In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from scipy.interpolate import CubicSpline
from scipy import signal
import os
import sklearn
import shutil
import os.path
import zipfile
from sklearn.preprocessing import MinMaxScaler

In [30]:
def windowing(data,seq_len,stride):
    date_time = []
    for i in range(0, data.shape[0] - seq_len + 1, stride):
        t_slice=  data[i : i + seq_len]
        data_time.append(t_slice)
    return data_time

In [36]:
def get_spectrogram(data_time,nfft,noverlap,nperseg):
    spectogram = []
    for i in range(data_time.shape[0]):
        f, t, specgram = signal.spectrogram( data_time[i,:],fs=250,nfft=nfft,noverlap=noverlap,nperseg=nperseg,mode="psd")
        specgram = np.log(specgram)
        specgram = (specgram - np.min(specgram)) / (np.max(specgram) - np.min(specgram))
        spectogram.append(specgram)
    return spectogram

In [ ]:
class WGAN(keras.Model):
    def __init__(
        self,
        discriminator,
        generator,
        latent_dim,
        discriminator_extra_steps=5,
        gp_weight=10.0,
    ):
        super(WGAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.d_steps = discriminator_extra_steps
        self.gp_weight = gp_weight

    def compile(self, d_optimizer, g_optimizer, d_loss_fn, g_loss_fn):
        super(WGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.d_loss_fn = d_loss_fn
        self.g_loss_fn = g_loss_fn

    def gradient_penalty(self, batch_size, real_images, fake_images):
        """ Calculates the gradient penalty.

        This loss is calculated on an interpolated image
        and added to the discriminator loss.
        """
        # Get the interpolated image
        alpha = tf.random.normal([batch_size, 1, 1, 1], 0.0, 1.0)
        diff = fake_images - real_images
        interpolated = real_images + alpha * diff

        with tf.GradientTape() as gp_tape:
            gp_tape.watch(interpolated)
            # 1. Get the discriminator output for this interpolated image.
            pred = self.discriminator(interpolated, training=True)

        # 2. Calculate the gradients w.r.t to this interpolated image.
        grads = gp_tape.gradient(pred, [interpolated])[0]
        # 3. Calculate the norm of the gradients.
        norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))
        gp = tf.reduce_mean((norm - 1.0) ** 2)
        return gp

    def train_step(self, real_images):
        if isinstance(real_images, tuple):
            real_images = real_images[0]

        # Get the batch size
        batch_size = tf.shape(real_images)[0]

        # For each batch, we are going to perform the
        # following steps as laid out in the original paper:
        # 1. Train the generator and get the generator loss
        # 2. Train the discriminator and get the discriminator loss
        # 3. Calculate the gradient penalty
        # 4. Multiply this gradient penalty with a constant weight factor
        # 5. Add the gradient penalty to the discriminator loss
        # 6. Return the generator and discriminator losses as a loss dictionary

        # Train the discriminator first. The original paper recommends training
        # the discriminator for `x` more steps (typically 5) as compared to
        # one step of the generator. Here we will train it for 3 extra steps
        # as compared to 5 to reduce the training time.
        for i in range(self.d_steps):
            # Get the latent vector
            random_latent_vectors = tf.random.normal(
                shape=(batch_size, self.latent_dim)
            )
            with tf.GradientTape() as tape:
                # Generate fake images from the latent vector
                fake_images = self.generator(random_latent_vectors, training=True)
                # Get the logits for the fake images
                fake_logits = self.discriminator(fake_images, training=True)
                # Get the logits for the real images
                real_logits = self.discriminator(real_images, training=True)

                # Calculate the discriminator loss using the fake and real image logits
                d_cost = self.d_loss_fn(real_img=real_logits, fake_img=fake_logits)
                # Calculate the gradient penalty
                gp = self.gradient_penalty(batch_size, real_images, fake_images)
                # Add the gradient penalty to the original discriminator loss
                d_loss = d_cost + gp * self.gp_weight

            # Get the gradients w.r.t the discriminator loss
            d_gradient = tape.gradient(d_loss, self.discriminator.trainable_variables)
            # Update the weights of the discriminator using the discriminator optimizer
            self.d_optimizer.apply_gradients(
                zip(d_gradient, self.discriminator.trainable_variables)
            )

        # Train the generator
        # Get the latent vector
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        with tf.GradientTape() as tape:
            # Generate fake images using the generator
            generated_images = self.generator(random_latent_vectors, training=True)
            # Get the discriminator logits for fake images
            gen_img_logits = self.discriminator(generated_images, training=True)
            # Calculate the generator loss
            g_loss = self.g_loss_fn(gen_img_logits)

        # Get the gradients w.r.t the generator loss
        gen_gradient = tape.gradient(g_loss, self.generator.trainable_variables)
        # Update the weights of the generator using the generator optimizer
        self.g_optimizer.apply_gradients(
            zip(gen_gradient, self.generator.trainable_variables)
        )
        return {"d_loss": d_loss, "g_loss": g_loss}

## Data Preprocessing

In [ ]:
df = pd.read_csv("/kaggle/input/bridge2-10files/BBD_bridge_2.csv")
# df = pd.read_csv("/kaggle/input/bridge2-10files/VVB_bridge_2.csv")
df_x = df[['x']].plot(title='x axis')
df_y = df[['y']].plot(title='y axis')
data = df[['x','y']].values
min_max_scaler = MinMaxScaler()
norm_data = min_max_scaler.fit_transform(data)
x = norm_data[:,0]
y = norm_data[:,1]

In [ ]:
# convert data to time series data 
seq_len = 500
stride = 76
x_time = windowing(x,seq_len,stride)
y_time = windowing(y,seq_len,stride)
x_time = np.array(x_time)
y_time = np.array(y_time)
print(x_time.shape)
print(y_time.shape)

In [52]:
# Convert x, y data to spectrogram
nfft = 1000
noverlap = 0
nperseg = 500
x_spec = get_spectrogram(x_time,nfft,noverlap,nperseg)
y_spec = get_spectrogram(y_time,nfft,noverlap,nperseg)
x_spec = np.array(x_spec)[:,:-1,:]
y_spec = np.array(y_spec)[:,:-1,:]
xy_spec = np.concatenate([x_spec, y_spec], axis=-1)
print(xy_spec.shape)
train_data = np.expand_dims(xy_spec,axis=-1)

(60, 500, 2)


## Train WGAN1

In [ ]:
# Create discriminator model
def make_discriminator(D,shape):
    input_layer = layers.Input(shape=shape)

    x = layers.Reshape((500,2))(input_layer)

    x = layers.Conv1D(D, 5, strides=2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv1D(D*2, 5, strides=2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv1D(D*4, 5, strides=2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv1D(D*8, 5, strides=2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv1D(D*16, 5, strides=2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(1)(x)
    
    discriminator = keras.models.Model(input_layer, x, name="discriminator")
    return discriminator

D = 128
shape = (500,2,1)
discriminator = make_discriminator(D,shape)
discriminator.summary()

In [ ]:
def make_generator(G,noise_dim):
    noise = layers.Input(shape=(noise_dim))
    x = layers.Dense(1 * 128 * G, use_bias=False)(noise)
    x = layers.Reshape((1, 128 * G))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.UpSampling1D(size=2)(x)
    x = layers.Conv1D(64 * G, 5, padding='same')(x)
    x = layers.Activation("relu")(x)

    x = layers.UpSampling1D(size=2)(x)
    x = layers.Conv1D(32 * G, 5, padding='same')(x)
    x = layers.Activation("relu")(x)

    x = layers.UpSampling1D(size=2)(x)
    x = layers.Conv1D(16 * G, 5, padding='same')(x)
    x = layers.Activation("relu")(x)

    x = layers.UpSampling1D(size=2)(x)
    x = layers.Conv1D(8 * G, 5, padding='same')(x)
    x = layers.Activation("relu")(x)

    x = layers.UpSampling1D(size=2)(x)
    x = layers.Conv1D(4 * G, 5, padding='same')(x)
    x = layers.Activation("relu")(x)

    x = layers.UpSampling1D(size=2)(x)
    x = layers.Conv1D(2 * G, 5, padding='same')(x)
    x = layers.Activation("relu")(x)

    x = layers.UpSampling1D(size=2)(x)
    x = layers.Conv1D(1 * G, 5, padding='same')(x)
    x = layers.Activation("relu")(x)

    x = layers.UpSampling1D(size=2)(x)
    x = layers.Conv1D(G, 5, padding='same')(x)
    x = layers.Activation("relu")(x)
    x = layers.Cropping1D(cropping=3)(x)

    x = layers.UpSampling1D(size=2)(x)
    x = layers.Conv1D(2, 5, padding='same')(x)
    x = layers.Activation("tanh")(x)

    x = layers.Reshape((500,2,1))(x)
 
    generator = keras.models.Model(noise, x, name="generator")
    return generator

noise_dim = 15
G = 32
generator = make_generator(G,noise_dim)
generator.summary()

In [ ]:
os.mkdir('/kaggle/working/loss_graph')
os.mkdir('/kaggle/working/spec_gen')
os.mkdir('/kaggle/working/disc_gen')

In [ ]:
class GANMonitor(keras.callbacks.Callback):
    def __init__(self, tries, num_img=6, latent_dim=noise_dim,plot_interval=500):
        self.num_img = num_img
        self.latent_dim = latent_dim
        self.plot_interval = plot_interval
        self.d_losses = []
        self.g_losses = []
        self.tries = tries

    def on_epoch_end(self, epoch, logs=None):
        self.d_losses.append(logs["d_loss"])
        self.g_losses.append(logs["g_loss"])
        if epoch >= 0:
            if epoch % self.plot_interval == 0:
                self.plot_losses(epoch)
                self.save_model(epoch)

    def plot_losses(self, epoch):
        plt.figure(figsize=(10, 5))
        plt.plot(self.g_losses, label="Generator Loss")
        plt.plot(self.d_losses, label="Discriminator Loss")
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.title("GAN Losses")
        plt.legend()
        plt.savefig('/kaggle/working/loss_graph/Disc_loss_%04d_epoch%04d.png' % (self.tries, epoch))

    def save_model(self, epoch):
        generator.save('/kaggle/working/spec_gen/generator_%04d_epoch%04d.keras' % (self.tries, epoch))
        discriminator.save('/kaggle/working/disc_gen/discriminator_%04d_epoch%04d.keras' % (self.tries, epoch))

# define optimizers and loss function 
generator_optimizer = keras.optimizers.Adam(
    learning_rate=0.0002, beta_1=0.5, beta_2=0.9
)
discriminator_optimizer = keras.optimizers.Adam(
    learning_rate=0.0002, beta_1=0.5, beta_2=0.9
)

def discriminator_loss(real_img, fake_img):
    real_loss = tf.reduce_mean(real_img)
    fake_loss = tf.reduce_mean(fake_img)
    return fake_loss - real_loss

def generator_loss(fake_img):
    return -tf.reduce_mean(fake_img)

# Set hyperparameter
epochs = 25000
tries = 9
batch_size = 10

# Instantiate callback
cbk = GANMonitor(tries, num_img=1, latent_dim=noise_dim,plot_interval=1000)

# Instantiate the WGAN model.
wgan = WGAN(
    discriminator=discriminator,
    generator=generator,
    latent_dim=noise_dim,
    discriminator_extra_steps=5,
)

# Compile the WGAN model.
wgan.compile(
    d_optimizer=discriminator_optimizer,
    g_optimizer=generator_optimizer,
    g_loss_fn=generator_loss,
    d_loss_fn=discriminator_loss,
)

# Start training the model.
history = wgan.fit(train_data, batch_size=batch_size, epochs=epochs, callbacks=[cbk])
